## Case Study - Danny's Diner

In [ ]:
import pandas as pd
import sqlite3
import sql
sql.style = 'plain'
%load_ext sql
connection=sqlite3.connect("question_bank.db")
print("Successfully connected to SQL database")
# Connect to SQLite
%sql sqlite:///question_bank.db
print("success")

In [ ]:
%%sql

CREATE TABLE sales (
  "customer_id" VARCHAR(1),
  "order_date" DATE,
  "product_id" INTEGER
);

INSERT INTO sales
  ("customer_id", "order_date", "product_id")
VALUES
  ('A', '2021-01-01', '1'),
  ('A', '2021-01-01', '2'),
  ('A', '2021-01-07', '2'),
  ('A', '2021-01-10', '3'),
  ('A', '2021-01-11', '3'),
  ('A', '2021-01-11', '3'),
  ('B', '2021-01-01', '2'),
  ('B', '2021-01-02', '2'),
  ('B', '2021-01-04', '1'),
  ('B', '2021-01-11', '1'),
  ('B', '2021-01-16', '3'),
  ('B', '2021-02-01', '3'),
  ('C', '2021-01-01', '3'),
  ('C', '2021-01-01', '3'),
  ('C', '2021-01-07', '3');
 

CREATE TABLE menu (
  "product_id" INTEGER,
  "product_name" VARCHAR(5),
  "price" INTEGER
);

INSERT INTO menu
  ("product_id", "product_name", "price")
VALUES
  ('1', 'sushi', '10'),
  ('2', 'curry', '15'),
  ('3', 'ramen', '12');
  

CREATE TABLE members (
  "customer_id" VARCHAR(1),
  "join_date" DATE
);

INSERT INTO members
  ("customer_id", "join_date")
VALUES
  ('A', '2021-01-07'),
  ('B', '2021-01-09');

#### Introduction
Danny seriously loves Japanese food so in the beginning of 2021, he decides to embark upon a risky venture and opens up a cute little restaurant that sells his 3 favourite foods: sushi, curry and ramen.

Danny’s Diner is in need of your assistance to help the restaurant stay afloat - the restaurant has captured some very basic data from their few months of operation but have no idea how to use their data to help them run the business.

#### Problem Statement
Danny wants to use the data to answer a few simple questions about his customers, especially about their visiting patterns, how much money they’ve spent and also which menu items are their favourite. Having this deeper connection with his customers will help him deliver a better and more personalised experience for his loyal customers.

He plans on using these insights to help him decide whether he should expand the existing customer loyalty program - additionally he needs help to generate some basic datasets so his team can easily inspect the data without needing to use SQL.

Danny has provided you with a sample of his overall customer data due to privacy issues - but he hopes that these examples are enough for you to write fully functioning SQL queries to help him answer his questions!

Danny has shared with you 3 key datasets for this case study:

* sales
* menu
* members

In [5]:
%%sql
SELECT * FROM sales

 * sqlite:///question_bank.db
Done.


customer_id,order_date,product_id
A,2021-01-01,1
A,2021-01-01,2
A,2021-01-07,2
A,2021-01-10,3
A,2021-01-11,3
A,2021-01-11,3
B,2021-01-01,2
B,2021-01-02,2
B,2021-01-04,1
B,2021-01-11,1


In [6]:
%%sql
SELECT * FROM menu

 * sqlite:///question_bank.db
Done.


product_id,product_name,price
1,sushi,10
2,curry,15
3,ramen,12


In [7]:
%%sql
SELECT * FROM members

 * sqlite:///question_bank.db
Done.


customer_id,join_date
A,2021-01-07
B,2021-01-09


#### Questions
* What is the total amount each customer spent at the restaurant?
* How many days has each customer visited the restaurant?
* What was the first item from the menu purchased by each customer?
* What is the most purchased item on the menu and how many times was it purchased by all customers?
* Which item was the most popular for each customer?
* Which item was purchased first by the customer after they became a member?
* Which item was purchased just before the customer became a member?
* What is the total items and amount spent for each member before they became a member?
* If each $1 spent equates to 10 points and sushi has a 2x points multiplier - how many points would each customer have?
* In the first week after a customer joins the program (including their join date) they earn 2x points on all items, not just sushi - how many points do customer A and B have at the end of January?

In [13]:
%%sql
-- What is the total amount each customer spent at the restaurant?
-- total amt, each customer -- Join sales and menu
-- QUERY

SELECT customer_id,sum(price) AS total_amt_each_customer
FROM sales s
LEFT JOIN menu m
ON s.product_id=m.product_id
GROUP BY s.customer_id


 * sqlite:///question_bank.db
Done.


customer_id,total_amt_each_customer
A,76
B,74
C,36


In [21]:
%%sql
-- How many days has each customer visited the restaurant?
-- count the no of days, each customer visited
-- QUERY

SELECT customer_id,count(distinct order_date) AS no_of_days_visited
FROM sales
GROUP BY customer_id

-- distinct dates, each customer has visited(customer A is visited two time on 2n Jan, we'll count only 1)

 * sqlite:///question_bank.db
Done.


customer_id,no_of_days_visited
A,4
B,6
C,2


In [30]:
%%sql
-- What was the first item from the menu purchased by each customer?
-- item, each customer

SELECT 
    s.customer_id,
    MIN(order_date) AS first_purchase_date,
    m.product_name
FROM sales s
LEFT JOIN menu m
ON s.product_id=m.product_id
GROUP BY customer_id

-- So first item - min date, each customer made a purchase

 * sqlite:///question_bank.db
Done.


customer_id,first_purchase_date,product_name
A,2021-01-01,sushi
B,2021-01-01,curry
C,2021-01-01,ramen


In [40]:
%%sql
-- What is the most purchased item on the menu and how many times was it purchased by all customers?

WITH temp AS (
    SELECT product_id, COUNT(product_id) AS no_of_purchases
    FROM sales
    GROUP BY product_id
)
SELECT t.product_id,t.no_of_purchases
FROM temp t
JOIN sales s
ON t.product_id=s.product_id
-- so, most purchased item on the menu is product_id=3, which is ramen

 * sqlite:///question_bank.db
Done.


product_id,no_of_purchases
1,3
2,4
2,4
3,8
3,8
3,8
2,4
2,4
1,3
1,3
